# IMPORTS

In [1]:
import csv
import io
import pydotplus
from IPython.display import display, Image
from rdflib.tools.rdf2dot import rdf2dot
from rdflib import Graph, Literal, Namespace, URIRef
from rdflib.namespace import DCTERMS, RDF, RDFS, SKOS, XSD, OWL
# from sklearn.datasets import fetch_20newsgroups
# newsgroups_train = fetch_20newsgroups(subset='train', shuffle = True)
# newsgroups_test = fetch_20newsgroups(subset='test', shuffle = True)
# import gensim
# import gensim.corpora as corpora
from pprint import pprint
# import pyLDAvis.gensim_models
# import pickle 
# import pyLDAvis

import nltk
#nltk.download('punkt')
import os
import nltk.corpus# sample text for performing tokenization
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords, wordnet
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer() 
from nltk.tokenize.treebank import TreebankWordDetokenizer
import spacy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from spacy.lang.en import English
from spacy import displacy
from collections import Counter
nlp = spacy.load('en_core_web_lg')
from spacy.matcher import Matcher
from spacy.tokens import Span
from spacy import displacy 
from nltk.util import ngrams
import string
import numpy as np
import inspect
from nltk.tag.perceptron import PerceptronTagger

from wordcloud import WordCloud

C:\Users\maily\Anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\Users\maily\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.o

In [2]:
pd.options.display.max_rows = 200
import decimal

# LOADING FILES

In [3]:
processes = []
root_directory = 'processes/'
for folder in os.listdir(root_directory):
    process_dir = root_directory + folder + '/'
    actual_process = []
    for filename in os.listdir(process_dir):
        with open(process_dir+filename, encoding="utf8") as file:
            actual_process.append(file.read())
    processes.append(actual_process)

# PREPROCESSING

In [4]:
def listToString(s): 
    
    # initialize an empty string
    str1 = "" 
    
    # traverse in the string  
    for ele in s: 
        str1 += ele+" "
    
    # return string  
    return str1 

def delete_number(Z):
    a = ['0','1','2','3','4','5','6','7','8','9',' 5' ]
    
    strings_copy = Z.copy()

    for i in np.arange(0,len(strings_copy)):
        if strings_copy[i][0] in a:
            Z.remove(strings_copy[i])
    return Z

def nettoyage(text):
    
    text= text.replace("•","")
    text= text.replace("pmi","")
    return text

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag in ['VERB','AUX']:
        return wordnet.VERB
    elif treebank_tag in ['NOUN','PROPN','PRON']:
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def segmentation(process):
    doc=nlp(process)
    for tok in doc: 
        print(tok.text, "-->",tok.dep_,"-->", tok.pos_)
        
def representation(process):
    doc=nlp(process) 
    displacy.render(doc, style='dep',jupyter=True)

In [5]:
def pos_lem_clean(process):
    
    all_sentences=sent_tokenize(process)

    postag_sentences=[]
    preprocessed=[]
    list_preprocessed=[]
    stop_words=set(stopwords.words("english"))
    list_empty=[]

    for sentence in all_sentences:

        doc = nlp(sentence)
        for tok in doc:
            tup=(tok.text,tok.pos_)
            postag_sentences.append(tup)
        list_empty.append(postag_sentences)
        postag_sentences=[]


    for sentence in list_empty:
        for tup in sentence:
            word=tup[0]
            tag=tup[1]
            type_word=get_wordnet_pos(tag)

            try:
                lem=lemmatizer.lemmatize(word,type_word)
                
                
                if lem !='' and lem not in stop_words and tag in ['VERB','NOUN','PROPN','PRON', 'AUX'] or lem == 'be':
                    preprocessed.append(lem.lower())


            except:
                pass
        list_preprocessed.append(preprocessed)
        preprocessed=[]
    return all_sentences, list_preprocessed

In [6]:
for p in range(len(processes)):
    for i in range(len(processes[p])):
        processes[p][i] = pos_lem_clean(nettoyage(processes[p][i]))

In [7]:
list_df = []
types = ["Inputs", "Others", "Outputs", "Tools"]
for process in processes:
    list_df_inputs_other_outputs_tools = []
    j=0
    for i in range(len(process)):
        df = pd.DataFrame(columns=['Sentence', 'Preprocessed'])
        df['Sentence'] = process[i][0]
        df['Preprocessed']= process[i][1]
        df['Type'] = types[j]
        list_df_inputs_other_outputs_tools.append(df)
        j += 1
    df_actual_process = pd.concat(list_df_inputs_other_outputs_tools, ignore_index='True')
    list_df.append(df_actual_process)

# Finding & Replacing synonyms

In [8]:
# df must have a column named : Preprocess (with capital P)
def find_synonyms(df):
    list_synonyms=[]
    list_words=[]
    lista=[]

    for sentence in df['Preprocessed']:
            for word in sentence:
                if word not in list_words:
                    list_words.append(word)
    for word in list_words:
        syns = wordnet.synsets(word)
        for syn in syns:
            for l in syn.lemmas():
                if l.name() not in lista:
                    lista.append(l.name())
        list_synonyms.append(lista)
        lista=[]
    
    return list_words, list_synonyms

def replace_synonyms(sentences, df_synonyms):
    checked_words = ['',' ']
    for sentence in sentences:
        for word in sentence:
            if word not in checked_words:
                synonyms = df_synonyms.loc[df_synonyms['Word'] == word]['Synonyms']
                if(len(synonyms != 0)):
                    word_synonyms = list(synonyms)[0]
                    checked_words.append(word)
                    for i in range(len(sentences)):
                        for j in range(len(sentences[i])):
                            if sentences[i][j] in word_synonyms:
                                sentences[i][j] = word
                
    return sentences

In [9]:
df_synonyms = pd.DataFrame(columns=['Word', 'Synonyms'])
df_synonyms['Word']=find_synonyms(pd.concat(list_df))[0]
df_synonyms['Synonyms']=find_synonyms(pd.concat(list_df))[1]
df_synonyms

,Word,Synonyms
0,element,"[component, constituent, element, factor, ingr..."
1,risk,"[hazard, jeopardy, peril, risk, endangerment, ..."
2,management,"[management, direction]"
3,plan,"[plan, program, programme, design, architectur..."
4,contribute,"[lend, impart, bestow, contribute, add, bring,..."
...,...,...
998,resultant,"[resultant, end_point, result, final_result, o..."
999,continue,"[continue, go_on, proceed, go_along, keep, car..."
1000,deem,"[deem, hold, view_as, take_for]"
1001,predefined,[]


In [10]:
for i in range(len(list_df)):
    clean_process = []
    clean_process = replace_synonyms(list(list_df[i]['Preprocessed']), df_synonyms)

    list_df[i]['Preprocessed'] = clean_process

In [11]:
list_df[0]

,Sentence,Preprocessed,Type
0,Key elements of the risk management plan that ...,"[element, risks, management, plan, contribute,...",Inputs
1,The cost management plan provides processes an...,"[be, management, plan, provide, process, contr...",Inputs
2,The schedule management plan provides insight ...,"[schedule, management, plan, provide, insight,...",Inputs
3,The quality management plan provides a baselin...,"[quality, management, plan, provide, baseline,...",Inputs
4,The human resource management plan provides gu...,"[resource, management, plan, provide, guidance...",Inputs
5,It can also contain roles and responsibilities...,"[it, control, role, responsibility, plan, orga...",Inputs
6,Project assumptions are found in the project s...,"[plan, assumptions, be, find, plan, scope, sta...",Inputs
7,Uncertainty in\nproject assumptions should be ...,"[uncertainty, plan, assumptions, be, measure, ...",Inputs
8,The WBS is a critical input to identifying ris...,"[wbs, be, input, identifies, risks, help, unde...",Inputs
9,Risks can be identified and subsequently track...,"[risks, be, identifies, track, summary, contro...",Inputs


In [12]:
# list_df[1]

In [13]:
# list_df[2]

In [14]:
# list_df[3]

In [15]:
# list_df[4]

In [16]:
# list_df[5]

# Spacy Rule Matching

<h3><h3>Extraction des mots composés et les relations (NN-NNN)<h3>

In [17]:
process_0 = list_df[0].copy()
for j in range(len(process_0)):
    process_0.iloc[j]['Preprocessed'] = ' '.join(list(process_0.iloc[j]['Preprocessed']))

process_1 = list_df[1].copy()
for j in range(len(list_df[1])):
    process_1.iloc[j]['Preprocessed'] = ' '.join(list(process_1.iloc[j]['Preprocessed']))

process_2 = list_df[2].copy()
for j in range(len(list_df[2])):
    process_2.iloc[j]['Preprocessed'] = ' '.join(list(process_2.iloc[j]['Preprocessed']))

process_3 = list_df[3].copy()    
for j in range(len(list_df[3])):
    process_3.iloc[j]['Preprocessed'] = ' '.join(list(process_3.iloc[j]['Preprocessed']))

process_4 = list_df[4].copy()    
for j in range(len(list_df[4])):
    process_4.iloc[j]['Preprocessed'] = ' '.join(list(process_4.iloc[j]['Preprocessed']))

process_5 = list_df[5].copy()    
for j in range(len(list_df[5])):
    process_5.iloc[j]['Preprocessed'] = ' '.join(list(process_5.iloc[j]['Preprocessed']))


In [18]:
combined_raw_texts = [' '.join(process_0['Sentence']), 
                      ' '.join(process_1['Sentence']), 
                      ' '.join(process_2['Sentence']), 
                      ' '.join(process_3['Sentence']), 
                      ' '.join(process_4['Sentence']), 
                      ' '.join(process_5['Sentence'])
                     ]

In [19]:
processes_copy = [process_0, process_1, process_2, process_3, process_4, process_5]
corpora = []
for process in processes_copy:
    preprocessed_text = []
    for i in range(len(process)):
        preprocessed_text.append(process.iloc[i]['Preprocessed'])
    preprocessed_combined_text = ' '.join(preprocessed_text) #COMBINED PREPROCESSED TEXT
    corpora.append(preprocessed_combined_text)

<h3>Named Entities Extraction<h3>

In [20]:
def entityExtraction(s):
    doc = nlp(s)
    d = []
    for ent in doc.ents:
        d.append((ent.label_, ent.text))
    df_entity = pd.DataFrame(d, columns = ('named entity', 'output'))
    return df_entity
df_raw_text_entity = entityExtraction(' '.join(combined_raw_texts))
df_raw_text_entity.loc[df_raw_text_entity['named entity'] == 'ORG'].reset_index()

,index,named entity,output
0,0,ORG,WBS
1,3,ORG,Project Risk Management
2,6,ORG,the Project Risk Management
3,10,ORG,Identify Risks
4,14,ORG,Delphi
5,15,ORG,Delphi
6,17,ORG,RBS
7,19,ORG,SWOT
8,20,ORG,SWOT
9,21,ORG,Project Risk Management


In [21]:
def relation_1(corpus):
    matcher_1 = Matcher(nlp.vocab)
    patterns_1 = [
        
        [{'POS': 'NOUN'}, {'POS': 'VERB'}, {'POS': 'NOUN'},{'POS': 'NOUN'}, {'POS': 'NOUN'}],
        [{'POS': 'NOUN'}, {'POS': 'VERB'}, {'POS': 'NOUN'},{'POS': 'NOUN'}],
        [{'POS': 'NOUN'}, {'POS': 'VERB'}, {'POS': 'NOUN'}],
        
        [{'POS': 'NOUN'}, {'POS': 'AUX'}, {'POS': 'NOUN'},{'POS': 'NOUN'}, {'POS': 'NOUN'}],
        [{'POS': 'NOUN'}, {'POS': 'AUX'}, {'POS': 'NOUN'},{'POS': 'NOUN'}],
        [{'POS': 'NOUN'}, {'POS': 'AUX'}, {'POS': 'NOUN'}],

        ]

    doc = nlp(corpus)
    matcher_1.add("id1", patterns_1)
    matches_1 = matcher_1(doc)
    matches_1 = set([doc[start:end].text for match_id, start, end in matches_1])
    matcher_1.remove("id1")
    matches_1




    relations_1 = []
    for relation in matches_1:
        relations_1.append(relation)
    liste_rights1=[]
    liste_lefts1=[]
    lefts1=[]
    rights1=[]
    verbs1=[]
    for text in relations_1:
        lefts=''
        rights=''
        lefts=text.split()[0]
        rights=text.split()[2:5]
        verb=text.split()[1]

        word=''

        liste_lefts1.append(lefts)
        liste_rights1.append(rights)
        verbs1.append(verb)

    for lefts in liste_lefts1:
        word=''
        for words in lefts:
            word = word + words + ' '
        lefts1.append(word)

    for rights in liste_rights1:
        word=''
        for words in rights:
            word = word + words + ' '
        rights1.append(word)


    df_relation_1 = pd.DataFrame(columns=['Relation', 'Lefts', 'Predicate', 'Rights'])
    df_relation_1['Relation']=relations_1
    df_relation_1['Lefts']= liste_lefts1
    df_relation_1['Rights']= rights1
    df_relation_1['Predicate']= verbs1
    
    return df_relation_1

In [22]:
def relation_2(corpus):
    matcher_2 = Matcher(nlp.vocab)
    patterns_2 = [
        
        [{'POS': 'NOUN'}, {'POS': 'NOUN'}, {'POS': 'VERB'}, {'POS': 'NOUN'},{'POS': 'NOUN'}, {'POS': 'NOUN'}],
        [{'POS': 'NOUN'}, {'POS': 'NOUN'}, {'POS': 'VERB'}, {'POS': 'NOUN'},{'POS': 'NOUN'}],
        [{'POS': 'NOUN'}, {'POS': 'NOUN'}, {'POS': 'VERB'}, {'POS': 'NOUN'}],
        
        [{'POS': 'NOUN'}, {'POS': 'NOUN'}, {'POS': 'AUX'}, {'POS': 'NOUN'},{'POS': 'NOUN'}, {'POS': 'NOUN'}],
        [{'POS': 'NOUN'}, {'POS': 'NOUN'}, {'POS': 'AUX'}, {'POS': 'NOUN'},{'POS': 'NOUN'}],
        [{'POS': 'NOUN'}, {'POS': 'NOUN'}, {'POS': 'AUX'}, {'POS': 'NOUN'}],

        ]

    doc = nlp(corpus)
    matcher_2.add("id", patterns_2)
    matches_2 = matcher_2(doc)
    matches_2 = set([doc[start:end].text for match_id, start, end in matches_2])
    matcher_2.remove("id")
    matches_2

    relations_2 = []
    for relation in matches_2:
        relations_2.append(relation)
    liste_rights2=[]
    liste_lefts2=[]
    lefts2=[]
    rights2=[]
    verbs2=[]
    for text in relations_2:
        lefts=''
        rights=''
        lefts=text.split()[:2]
        rights=text.split()[3:]
        verb=text.split()[2]

        word=''

        liste_lefts2.append(lefts)
        liste_rights2.append(rights)
        verbs2.append(verb)
    for lefts in liste_lefts2:
        word=''
        for words in lefts:
            word = word + words + ' '
        lefts2.append(word)

    for rights in liste_rights2:
        word=''
        for words in rights:
            word = word + words + ' '
        rights2.append(word)


    df_relation_2 = pd.DataFrame(columns=['Relation', 'Lefts', 'Predicate', 'Rights'])
    df_relation_2['Relation']=relations_2
    df_relation_2['Lefts']= lefts2
    df_relation_2['Rights']= rights2
    df_relation_2['Predicate']= verbs2
    
    return df_relation_2

In [23]:
def relation_3(corpus):
    matcher_3 = Matcher(nlp.vocab)
    patterns_3 = [

        [{'POS': 'NOUN'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}, {'POS': 'VERB'}, {'POS': 'NOUN'},{'POS': 'NOUN'}, {'POS': 'NOUN'}],
        [{'POS': 'NOUN'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}, {'POS': 'VERB'}, {'POS': 'NOUN'},{'POS': 'NOUN'}],
        [{'POS': 'NOUN'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}, {'POS': 'VERB'}, {'POS': 'NOUN'}],
        
        [{'POS': 'NOUN'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}, {'POS': 'AUX'}, {'POS': 'NOUN'},{'POS': 'NOUN'}, {'POS': 'NOUN'}],
        [{'POS': 'NOUN'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}, {'POS': 'AUX'}, {'POS': 'NOUN'},{'POS': 'NOUN'}],
        [{'POS': 'NOUN'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}, {'POS': 'AUX'}, {'POS': 'NOUN'}],

        ]

    doc = nlp(corpus)
    matcher_3.add("id3", patterns_3)
    matches_3 = matcher_3(doc)
    matches_3 = set([doc[start:end].text for match_id, start, end in matches_3])
    matcher_3.remove("id3")
    matches_3


    relations_3 = []
    for relation in matches_3:
        relations_3.append(relation)
    # relations_3   

    liste_rights3=[]
    liste_lefts3=[]
    lefts3=[]
    rights3=[]
    verbs3=[]
    for text in relations_3:
        lefts=''
        rights=''
        lefts=text.split()[:3]
        rights=text.split()[4:]
        verb=text.split()[3]

        word=''

        liste_lefts3.append(lefts)
        liste_rights3.append(rights)
        verbs3.append(verb)
    for lefts in liste_lefts3:
        word=''
        for words in lefts:
            word = word + words + ' '
        lefts3.append(word)

    for rights in liste_rights3:
        word=''
        for words in rights:
            word = word + words + ' '
        rights3.append(word)


    df_relation_3 = pd.DataFrame(columns=['Relation', 'Lefts', 'Predicate', 'Rights'])
    df_relation_3['Relation']=relations_3
    df_relation_3['Lefts']= lefts3
    df_relation_3['Rights']= rights3
    df_relation_3['Predicate']= verbs3
    
    return df_relation_3

In [24]:
relations_by_process = []
for corpus in corpora:
    rel1 = relation_1(corpus)
    rel2 = relation_2(corpus)
    rel3 = relation_3(corpus)
    all_rels = pd.concat([rel1, rel2, rel3], ignore_index='True')
    relations_by_process.append(all_rels)

<h3>Removing useless relations<h3>

In [25]:
def create_verification_text(raw_process_text):
    verification_text = []
    nlp_process_text = nlp(raw_process_text.strip().lower())
    for word in nlp_process_text:
        verification_text.append(word.lemma_)
    return verification_text

def check_term_2(term, verification_text):
    i=0
    while i + 1 < len(verification_text): 
        combination = (verification_text[i]+' '+verification_text[i+1]).strip()
        if combination == term:
            return True
        i+=1
    return False

def check_term_3(term, verification_text):
    i=0
    while i + 2 < len(verification_text): 
        combination = (verification_text[i]+' '+verification_text[i+1]+' '+verification_text[i+2]).strip()
        if combination == term:
            return True
        i+=1
    return False

In [26]:
process_num=0
for k in range(len(relations_by_process)):
    combined_raw_text = combined_raw_texts[process_num]
    verification_text = create_verification_text(combined_raw_text)
    copy = relations_by_process[k].copy()
    for i in range(len(copy)):
        row= copy.iloc[i]
        if len(row['Lefts'].split()) == 3:
            lefts_relavance = check_term_3(row['Lefts'].strip(), verification_text)
        elif len(row['Lefts'].split()) == 2:
            lefts_relavance = check_term_2(row['Lefts'].strip(), verification_text)
        else:
            lefts_relavance = True

        if len(row['Rights'].split()) == 3:
            rights_relavance = check_term_3(row['Rights'].strip(), verification_text)
        elif len(row['Rights'].split()) == 2:
            rights_relavance = check_term_2(row['Rights'].strip(), verification_text)
        else:
            rights_relavance = True
        if not(lefts_relavance and rights_relavance):
            relations_by_process[k].drop(i,axis=0, inplace=True)
           
    process_num += 1

In [27]:
####################################### LEMMATIZING DES PREDICATES ##########################################

for i in range(len(relations_by_process)):
    for j in range(len(relations_by_process[i])):
        lem=lemmatizer.lemmatize(relations_by_process[i].iloc[j]['Predicate'],get_wordnet_pos('VERB'))
        relations_by_process[i].iloc[j]['Predicate'] =  lem
        
############################################ RESETTING INDEXES ##############################################

for i in range(len(relations_by_process)):
    relations_by_process[i].reset_index(inplace=True, drop=True)

In [28]:
process_names = ['identify_risk', 'monitor_and_control_risks', 
                 'perform_qualitative_risk_analysis', 
                 'perform_quantitative_risk_analysis', 
                 'plan_risk_management', 
                 'plan_risk_responses'
                ]
for i in range(len(relations_by_process)):
    relations_by_process[i]['Process'] = process_names[i]

In [29]:
# relations_by_process[5]

In [30]:
# relations_by_process[4]

In [31]:
# relations_by_process[3]

In [32]:
# relations_by_process[2]

In [33]:
# relations_by_process[1]

In [34]:
relations_by_process[0]

,Relation,Lefts,Predicate,Rights,Process
0,risks express element,risks,express,element,identify_risk
1,risks know quality,risks,know,quality,identify_risk
2,order expose area,order,expose,area,identify_risk
3,detail appear checklist,detail,appear,checklist,identify_risk
4,risks plan document,risks,plan,document,identify_risk
...,...,...,...,...,...
216,management process identifies risks,management process,identify,risks,identify_risk
217,management process risks identification,management process,risk,identification,identify_risk
218,identification technique be role,identification technique,be,role,identify_risk
219,schedule management plan provide insight,schedule management plan,provide,insight,identify_risk


In [61]:
project_risk_management_relations = pd.concat(relations_by_process, ignore_index=True)

<h3>Adding relations that correspond to extracted named entities<h3>

In [63]:
df = pd.DataFrame({'Lefts':['wbs'], 
                  'Predicate':['be'], 
                  'Rights':['input'], 
                  'Process': ['identify_risks']}, index=None)
pd.concat([project_risk_management_relations, df])

df = pd.DataFrame({'Lefts':['delphi_technique'], 
                  'Predicate':['be'], 
                  'Rights':['hierarchical_representation'], 
                  'Process': ['identify_risks']}, index=None)
pd.concat([project_risk_management_relations, df])

df = pd.DataFrame({'Lefts':['rbs'], 
                  'Predicate':['be'], 
                  'Rights':['input'], 
                  'Process': ['identify_risks']}, index=None)
pd.concat([project_risk_management_relations, df])
project_risk_management_relations.reset_index(inplace=True, drop=True)
project_risk_management_relations['Predicate'].replace({'be':'is_a'}, inplace = True)

In [66]:
project_risk_management_relations.loc[project_risk_management_relations['Predicate'] == 'is_a']

,Relation,Lefts,Predicate,Rights,Process
12,risks be entry,risks,is_a,entry,identify_risk
14,process be focus,process,is_a,focus,identify_risk
25,extent be fact,extent,is_a,fact,identify_risk
26,process be conduct,process,is_a,conduct,identify_risk
27,checklist be review,checklist,is_a,review,identify_risk
28,time be quality,time,is_a,quality,identify_risk
33,bias be charter,bias,is_a,charter,identify_risk
34,process be repeat,process,is_a,repeat,identify_risk
37,process be record,process,is_a,record,identify_risk
40,document be measure,document,is_a,measure,identify_risk


# CLASSES, SUB-CLASSES AND INDIVIDUALS EXTRACTION

<h3>Classes Extraction<h3>

In [38]:
############################################## Extracting Classes #################################################

list_classes_by_process = []
for name in process_names:
    ################# Extracting classes from the left side of the relations #################

    left_classes = project_risk_management_relations.loc[project_risk_management_relations["Process"] == name].copy()
    left_classes.reset_index(inplace=True, drop=True)
    copy = left_classes.copy()
    for i in range(len(copy)):
        copy.iloc[i]['Lefts'] = copy.iloc[i]['Lefts'].strip() 
        if " " in copy['Lefts'].iloc[i]:
            left_classes.drop(i, axis=0, inplace = True)
    left_classes = left_classes['Lefts'].values

    ################# Extracting classes from the right side of the relations #################

    right_classes = project_risk_management_relations.loc[project_risk_management_relations["Process"] == name].copy()
    right_classes.reset_index(inplace=True, drop=True)
    copy = right_classes.copy()
    for i in range(len(copy)):
        copy.iloc[i]['Rights'] = copy.iloc[i]['Rights'].strip()
        if " " in copy['Rights'].iloc[i]:
            right_classes.drop(i, axis=0, inplace = True)
    right_classes = right_classes['Rights'].values

    ################## Combining and stripping classes in a single array ######################

    classes = list(set(list(left_classes) + list(right_classes)))
    for i in range(len(classes)):
        classes[i] = classes[i].strip()
    list_classes_by_process.append(classes)

In [39]:
classes_by_process = []
i=0
for classes in list_classes_by_process:
    df = pd.DataFrame(columns=['Classes', 'Process'])
    df['Classes'] = classes
    df['Process'] = process_names[i]
    classes_by_process.append(df)
    i += 1

<h3>Sub-classes Extraction<h3>

In [40]:
for k in range(len(classes_by_process)):
    subclasses = []
    df = relations_by_process[k].copy()
    for c in classes_by_process[k]['Classes']:
        left_subclasses = []
        for i in range(len(df)):
            term = df.iloc[i]['Lefts'].strip()
#             if c in term and term != c and not(term.startswith(c)):
            if term.endswith((" "+c)) and not(term.startswith(c)):
                left_subclasses.append(term)


        right_subclasses = []
        for i in range(len(df)):
            term = df.iloc[i]['Rights'].strip()
#             if c in term and term != c and not(term.startswith(c)):
            if term.endswith((" "+c)) and not(term.startswith(c)):
                right_subclasses.append(term)

            
        subclasses.append(list(set(list(left_subclasses)+list(right_subclasses))))
    classes_by_process[k]['Subclasses'] = subclasses
    classes_by_process[k]['Process'] = process_names[k]
    print("process "+str(k)+" : OK !")

process 0 : OK !
process 1 : OK !
process 2 : OK !
process 3 : OK !
process 4 : OK !
process 5 : OK !


In [41]:
project_risk_management_classes = pd.concat(classes_by_process, ignore_index=True)

In [42]:
final_project_risk_management_classes = pd.DataFrame(columns = ['Classes', 'Process', 'Subclasses'])
for i in range(len(project_risk_management_classes)):
    old_subclasses = project_risk_management_classes.iloc[i]['Subclasses']
    new_subclasses = []
    
    for j in range(len(old_subclasses)):
        new_subclasses.append(old_subclasses[j].strip().replace(' ', '_'))
        
    df = pd.DataFrame({'Classes':[project_risk_management_classes.iloc[i]['Classes']], 
                      'Process':[project_risk_management_classes.iloc[i]['Process']], 
                      'Subclasses':[new_subclasses]})
    final_project_risk_management_classes = pd.concat([final_project_risk_management_classes, df])

<h3>Cleaning classes-subclasses Dataset

In [43]:
for i in range(len(final_project_risk_management_classes)):
    final_project_risk_management_classes.iloc[i]['Subclasses'] = listToString(final_project_risk_management_classes.iloc[i]['Subclasses'])
final_project_risk_management_classes.drop_duplicates(subset=None, keep='first', inplace=True)

In [44]:
for i in range(len(final_project_risk_management_classes)):
    final_project_risk_management_classes.iloc[i]['Subclasses'] = final_project_risk_management_classes.iloc[i]['Subclasses'].split(" ")
for i in range(len(final_project_risk_management_classes)):
    if " " in final_project_risk_management_classes.iloc[i]['Subclasses']:
        final_project_risk_management_classes.iloc[i]['Subclasses'] = final_project_risk_management_classes.iloc[i]['Subclasses'].remove(" ")
    

In [45]:
final_project_risk_management_classes.reset_index(inplace=True, drop=True)
final_project_risk_management_classes

,Classes,Process,Subclasses
0,people,identify_risk,[]
1,likelihood,identify_risk,[]
2,way,identify_risk,[]
3,effectiveness,identify_risk,[]
4,sources,identify_risk,[]
...,...,...,...
699,log,plan_risk_responses,[]
700,manager,plan_risk_responses,[]
701,seller,plan_risk_responses,[]
702,buy,plan_risk_responses,[]


<h3>Extracting Individuals<h3>

In [46]:
indiv_rels = project_risk_management_relations.loc[project_risk_management_relations['Predicate'] == 'is_a'].copy() ###### relations that have "is_a" as predicate
classes = final_project_risk_management_classes.copy() ########### all classes
list_df_indivs_by_process = [] ########## a list of dataframes where each list contains classes and their individuals for each process (one df for each process)

for process in process_names:
    df_indivs_by_process = pd.DataFrame(columns= ['Classes', 'Process','Individuals']) ##### instancing current processe's df
    classes_by_process = classes.loc[classes['Process'] == process]['Classes'].drop_duplicates().values ######## the classes that correspond to the current process
    indiv_rels_by_process = indiv_rels.loc[indiv_rels['Process'] == process] ####### relations that have "is_a" as predicate and that correspond to the current process
    
    #------------ Removing Extra spaces ---------------------------------------------------------#
    for k in range(len(indiv_rels_by_process)):
        indiv_rels_by_process.iloc[k]['Rights'] = indiv_rels_by_process.iloc[k]['Rights'].strip()
        indiv_rels_by_process.iloc[k]['Lefts'] = indiv_rels_by_process.iloc[k]['Lefts'].strip() 
    #--------------------------------------------------------------------------------------------#
    
    for cls in classes_by_process: ##### iterating over classes of the current process
        indivs_by_cls = indiv_rels_by_process.loc[indiv_rels_by_process['Rights'] == cls]['Lefts'].values ##### storing current classe's indivuduals in a list
        
        #--------- Replacing Spaces -------------------------# 
        for i in range(len(indivs_by_cls)):
            indivs_by_cls[i] = indivs_by_cls[i].replace(' ', '_') 
        #----------------------------------------------------#
        
        df = pd.DataFrame({'Classes':[cls], 
                           'Process':[process], 
                           'Individuals':[indivs_by_cls]
                          }, index=None)
        df_indivs_by_process = pd.concat([df_indivs_by_process,df])##### adding the triplet row ('class' -> 'process' -> 'individual') to df_indivs_by_process described above
        
    list_df_indivs_by_process.append(df_indivs_by_process) ##### adding df_indivs_by_process to list_df_indivs_by_process described above


In [47]:
project_risk_management_individuals = pd.concat(list_df_indivs_by_process)
project_risk_management_individuals.reset_index(inplace = True, drop = True)

In [48]:
project_risk_management_individuals

,Classes,Process,Individuals
0,people,identify_risk,[]
1,likelihood,identify_risk,[]
2,way,identify_risk,[technique]
3,effectiveness,identify_risk,[]
4,sources,identify_risk,[]
...,...,...,...
699,log,plan_risk_responses,[]
700,manager,plan_risk_responses,[]
701,seller,plan_risk_responses,[]
702,buy,plan_risk_responses,[]


In [49]:
final_project_risk_management_classes['Individuals'] = project_risk_management_individuals['Individuals']
for i in range(len(final_project_risk_management_classes)):
    for sc in final_project_risk_management_classes.iloc[i]['Subclasses']:
        if sc in final_project_risk_management_classes.iloc[i]['Individuals']:
            final_project_risk_management_classes.iloc[i]['Subclasses'].remove(sc)

In [50]:
final_project_risk_management_classes

,Classes,Process,Subclasses,Individuals
0,people,identify_risk,[],[]
1,likelihood,identify_risk,[],[]
2,way,identify_risk,[],[technique]
3,effectiveness,identify_risk,[],[]
4,sources,identify_risk,[],[]
...,...,...,...,...
699,log,plan_risk_responses,[],[]
700,manager,plan_risk_responses,[],[]
701,seller,plan_risk_responses,[],[]
702,buy,plan_risk_responses,[],[]


In [67]:
final_project_risk_management_classes.to_csv('project_risk_management_classes.csv')
project_risk_management_relations.to_csv('project_risk_management_relations.csv')

# RDF GRAPH CONSTRUCTION

In [52]:
############################# REMOVING SPACES FROM THE RELATIONS DATAFRAME ################################

for i in range(len(relations_by_process)):
    
    for j in range(len(relations_by_process[i])):

        lefts = relations_by_process[i].iloc[j]['Lefts']
        predicate = relations_by_process[i].iloc[j]['Predicate']  
        rights = relations_by_process[i].iloc[j]['Rights']

        relations_by_process[i]['Lefts'].replace({lefts : lefts.strip().replace(' ','_')}, inplace=True)
        relations_by_process[i]['Predicate'].replace({predicate : predicate.strip().replace(' ','_')}, inplace=True)
        relations_by_process[i]['Rights'].replace({rights : rights.strip().replace(' ','_')}, inplace=True)
        
project_risk_management_relations = pd.concat(relations_by_process, ignore_index=True)

In [110]:
def visualize(g):
    stream = io.StringIO()
    rdf2dot(g, stream, opts = {display})
    dg = pydotplus.graph_from_dot_data(stream.getvalue())
#     png = dg.create_png()
#     display(Image(png))
    dg.write_png('graph.png')

In [111]:
g=Graph()
g.bind("owl",OWL)
g.bind("pr",'http://www.example.org/')
ns_url='http://www.example.org/'
g.add((URIRef('http://www.example.org/'),RDF.type,OWL.Ontology))

<Graph identifier=N2a2b4a3d803b46a0adae59c6842cc912 (<class 'rdflib.graph.Graph'>)>

<h3>Adding Classes, Subclasses and Individuals<h3>

In [112]:
df = final_project_risk_management_classes.copy()
for process in process_names:
    p =  URIRef('http://www.example.org/' + process)
    uri = 'http://www.example.org/' + process + "/"
    g.add((p, RDF.type, OWL.Class))
    
    for c in df.loc[df['Process'] == process]['Classes']:
        cl = URIRef(uri + c)
        g.add((cl,RDFS.subClassOf, p))
        all_sc = list(df.loc[(df['Classes'] == c) & (df['Process'] == process)]['Subclasses'])[0]
        all_indivs = list(df.loc[(df['Classes'] == c) & (df['Process'] == process)]['Individuals'])[0]
        print(c +" : ")
        print(all_sc)

        for sc in all_sc:
            if sc:

                scl = URIRef(str(uri)+str(sc))
                g.add((scl, RDFS.subClassOf, cl))
                    
        for indiv in all_indivs:
            if indiv:
                individ = URIRef(str(uri)+str(indiv))
                g.add((individ, RDF.type, cl))
        for indiv in all_indivs:
            if indiv:
                individ = URIRef(str(uri)+str(indiv))
                g.add((individ, RDF.type, OWL.NamedIndividual))

people : 
['']
likelihood : 
['']
way : 
['']
effectiveness : 
['']
sources : 
['']
study : 
['industry_study', '']
conduct : 
['']
opportunity : 
['']
condition : 
['']
risks : 
['']
categories : 
['']
number : 
['']
repeat : 
['']
activity : 
['management_activity', '']
technique : 
['creativity_technique', 'diagramming_technique', 'identification_technique', 'assessment_technique', '']
checklist : 
['']
measure : 
['']
stakeholder : 
['']
ownership : 
['']
assignment : 
['']
iteration : 
['']
control : 
['process_control', '']
change : 
['exposure_change', '']
time : 
['']
organization : 
['']
assessment : 
['']
base : 
['']
order : 
['']
framework : 
['']
analysis : 
['']
help : 
['']
guidance : 
['']
ideas : 
['']
diagramming : 
['']
relationship : 
['']
result : 
['']
influence : 
['']
process : 
['management_process', '']
role : 
['']
schedule : 
['']
management : 
['quality_management', 'schedule_management', '']
nominate : 
['']
owner : 
['']
plan : 
['quality_management_plan'

could : 
['update_could', '']
tolerance : 
['']
uncertainty : 
['way_uncertainty', '']
scope : 
['']
bar : 
['']
variation : 
['']
point : 
['reference_point', '']
chapter : 
['']
collection : 
['']
% : 
['']
database : 
['']
tornado : 
['']
future : 
['']
extent : 
['']
reference : 
['']
experience : 
['']
industry : 
['']
budget : 
['']
evaluate : 
['']
repeat : 
['']
questionnaire : 
['']
workshop : 
['']
sensitivity : 
['']
reason : 
['']
importance : 
['']
impact : 
['']
provide : 
['']
contingency : 
['']
results : 
['']
taking : 
['']
completion : 
['']
performance : 
['']
urgency : 
['']
approach : 
['analysis_approach', '']
context : 
['management_context', '']
thresholds : 
['']
criteria : 
['']
content : 
['']
feasibility : 
['']
audit : 
['']
templates : 
['']
creation : 
['']
lessons : 
['']
size : 
['']
risks : 
['']
attitude : 
['']
escalation : 
['']
method : 
['monitoring_method', '']
expectation : 
['']
training : 
['']
activity : 
['execution_activity', 'management_a

<h3>Adding Object Properties and exporting .nt Graph<h3>

In [113]:
for i in range(len(project_risk_management_relations)):
    relation = project_risk_management_relations.iloc[i]
    uri = 'http://www.example.org/' + relation['Process'] + "/"
    prop = URIRef(uri + relation['Predicate'])
    subj = URIRef(uri + relation['Lefts'])
    obj = URIRef(uri + relation['Rights'])
    g.add((prop,RDF.type, OWL.ObjectProperty))
    g.add((prop,RDFS.domain, subj))
    g.add((prop, RDFS.range,obj))
    
g.serialize(destination='RDF.nt', format='nt')

<Graph identifier=N2a2b4a3d803b46a0adae59c6842cc912 (<class 'rdflib.graph.Graph'>)>